In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import labolibrary as labo
import pickle
from sklearn.preprocessing import RobustScaler,PowerTransformer
from keras.models import Sequential
from keras.layers import LSTM, Dense

d:\Dropbox\Python\LaboIII\labo3-2024v\.venv\lib\site-packages\dask\dataframe\__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


In [2]:

#DATOS_DIR = '~/buckets/b1/datasets/'
DATOS_DIR = '../data/'      

scalers = {}
# Function to center, scale, and return a series
def scale_group(group):
    scaler = RobustScaler()
    #scaler = PowerTransformer()
    scaled_values = scaler.fit_transform(group.values.reshape(-1, 1)).flatten()
    scalers[group.name] = scaler  # Store the scaler for this group
    return pd.Series(scaled_values, index=group.index, name=group.name)

# Function to inverse transform (de-scale) and decenter, and return a series
def inverse_scale_group(group):
    group_name = group.name
    scaler = scalers[group_name]
    inversed_centered_values = scaler.inverse_transform(group.values.reshape(-1, 1)).flatten()
    original_values = inversed_centered_values
    return pd.Series(original_values, index=group.index, name=group_name)


In [3]:

# Leer datos
#df_final = pd.read_parquet(DATOS_DIR+'FE_dataset-CARLA.parquet') 
#import dask.dataframe as dd
df_final = pd.read_parquet(DATOS_DIR+'FE_05_dataset.parquet') 
#df_final = df_final.compute()

df_final.columns = df_final.columns.str.replace(' ', '_').str.replace(r'[^A-Za-z0-9_]', '', regex=True)




In [ ]:

## ESTO ES UN HACK PORQUE ESTA MAL ARMADA LA FUNCIÓN DE ESCALADO Y SOLO GUARDA LOS DATOS DE LA ÜLTIMA APLICACIÓN


with open(DATOS_DIR + '/scalers.pkl', 'rb') as file:
    scalers = pickle.load(file)

df_test = df_final[(df_final['product_id'] == 21276) & (df_final['customer_id'] == 10029)][['product_id','tn', 'weight']]
df_test['tn'] = df_test.groupby('product_id')['tn'].transform(inverse_scale_group)

df_test

In [ ]:

### Filtrar datos
#df_true = df_final.loc['2019-12-01':'2020-01-01']
df_final = df_final.loc['2018-01-01':'2019-11-01']


In [5]:

#Filtro test
df_final = df_final[df_final['product_id'] < 20013]

weight= df_final[['weight','product_id','customer_id']]
#df_final.drop(columns=['weight'], inplace=True)
#filtered_weight = weight[weight['weight'] != 0]
#weight_mean = filtered_weight.groupby('product_id')['weight'].mean().to_dict()
#weight['weight'] = weight['product_id'].map(weight_mean)

weight = weight.groupby(['product_id','customer_id'])['weight'].mean()
weight = weight.reset_index()
df_final = df_final.drop(columns=['weight'])

In [6]:
# Correr Modelo
params={
        'boosting_type': 'gbdt',
        'objective': 'Regression',
        'metric':'rmse',
        'verbose': -1,
        #'n_jobs': -1,
        'seed': 10000079,
        #'learning_rate': 0.2,
        #'bagging_fraction': 0.85,
        #'bagging_freq': 1, 
        #'colsample_bytree': 0.85,
        #'colsample_bynode': 0.85,
        #'min_data_per_leaf': 25,
        #'num_leaves': 200,
        #'lambda_l1': 0.5,
        #'lambda_l2': 0.5
}

predictions_all = pd.DataFrame(columns=['tn'])
products = df_final['product_id'].unique()
tot = len(products)
nro = 0
for producto in products:
    print(f'Fitting and predicting for product_id: {producto}')
    # Filtrar los datos del producto
    df_producto = df_final[df_final['product_id'] == producto]
    
    product_weights = pd.merge(df_producto.reset_index(), weight, on=['product_id', 'customer_id'], how='left')
    product_weights.set_index('periodo', inplace=True)
    weight_p = product_weights['weight']
   
    
    # Prepare data for LSTM on tn_2 only
    X = df_producto[['tn_2']].values.astype('float32')
    y = df_producto['tn_2'].values.astype('float32')
    X = X.reshape((X.shape[0], 1, X.shape[1]))
    #######################################################    
    # Define LSTM model
    model = Sequential()
    model.add(LSTM(50, input_shape=(X.shape[1], X.shape[2])))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(optimizer='adam', loss='binary_crossentropy')

    # Train LSTM model
    model.fit(X, y, epochs=10, batch_size=32, verbose=1)

    # Extract features from LSTM
    features = model.predict(X)

    # Prepare data for LightGBM
    # Convert LSTM features to DataFrame
    features_df = pd.DataFrame(features, index=df_producto.index, columns=[f'lstm_feature_{i}' for i in range(features.shape[1])])
    df_producto = pd.concat([df_producto, features_df], axis=1)
    df_producto['tn_2'] = y
    
    #############################################################################  
    #### Agrupar y escalar
   
    model, average_metric = labo.train_lightgbm_model(df_producto,params,metric='rmse',
                                                      weights=weight_p)
    print("Overall rmse metric: ", average_metric)
    # Predict values for the entire dataset using the trained models
    # Prepare last data points for prediction
    last_data_points = df_producto[df_producto.index == df_producto.index.max()].copy()
    last_data_points.drop(columns=['tn_2'], inplace=True)
    # Predict the next month's value using the trained model
    predictions = labo.predict_next_month(model, last_data_points)
    preds = predictions.groupby('product_id')['tn_2'].transform(inverse_scale_group)
    predictions['tn'] = preds
    predictions.drop(columns=['tn_2'], inplace=True)
    predictions = predictions.reset_index()
    predictions =  predictions.groupby('product_id')['tn'].sum()
    predictions.columns = ['product_id', 'tn']
    predictions_all = pd.concat([predictions_all, predictions])
    print(predictions_all[-1:])


Fitting and predicting for product_id: 20011
Epoch 1/10


d:\Dropbox\Python\LaboIII\labo3-2024v\.venv\lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


305/305 ━━━━━━━━━━━━━━━━━━━━ 2s 799us/step - loss: -22.6840
Epoch 2/10
305/305 ━━━━━━━━━━━━━━━━━━━━ 0s 795us/step - loss: -116.5673
Epoch 3/10
305/305 ━━━━━━━━━━━━━━━━━━━━ 0s 920us/step - loss: -192.7872
Epoch 4/10
305/305 ━━━━━━━━━━━━━━━━━━━━ 0s 871us/step - loss: -238.1887
Epoch 5/10
305/305 ━━━━━━━━━━━━━━━━━━━━ 0s 719us/step - loss: -298.4529
Epoch 6/10
305/305 ━━━━━━━━━━━━━━━━━━━━ 0s 749us/step - loss: -322.0608
Epoch 7/10
305/305 ━━━━━━━━━━━━━━━━━━━━ 0s 812us/step - loss: -390.3206
Epoch 8/10
305/305 ━━━━━━━━━━━━━━━━━━━━ 0s 883us/step - loss: -430.7082
Epoch 9/10
305/305 ━━━━━━━━━━━━━━━━━━━━ 0s 939us/step - loss: -554.3570
Epoch 10/10
305/305 ━━━━━━━━━━━━━━━━━━━━ 0s 737us/step - loss: -600.3226
305/305 ━━━━━━━━━━━━━━━━━━━━ 0s 942us/step
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[62]	validation's rmse: 19.8035
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[42]	validation's rmse: 

C:\Users\Admin\AppData\Local\Temp\ipykernel_23064\1599199677.py:75: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predictions_all = pd.concat([predictions_all, predictions])
C:\Users\Admin\AppData\Local\Temp\ipykernel_23064\1599199677.py:75: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  predictions_all = pd.concat([predictions_all, predictions])
d:\Dropbox\Python\LaboIII\labo3-2024v\.venv\lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using S

347/347 ━━━━━━━━━━━━━━━━━━━━ 1s 674us/step - loss: -14.9116
Epoch 2/10
347/347 ━━━━━━━━━━━━━━━━━━━━ 0s 710us/step - loss: -68.8717
Epoch 3/10
347/347 ━━━━━━━━━━━━━━━━━━━━ 0s 750us/step - loss: -111.4121
Epoch 4/10
347/347 ━━━━━━━━━━━━━━━━━━━━ 0s 850us/step - loss: -173.2175
Epoch 5/10
347/347 ━━━━━━━━━━━━━━━━━━━━ 0s 696us/step - loss: -206.7426
Epoch 6/10
347/347 ━━━━━━━━━━━━━━━━━━━━ 0s 794us/step - loss: -217.9852
Epoch 7/10
347/347 ━━━━━━━━━━━━━━━━━━━━ 0s 727us/step - loss: -270.0739
Epoch 8/10
347/347 ━━━━━━━━━━━━━━━━━━━━ 0s 710us/step - loss: -339.3842
Epoch 9/10
347/347 ━━━━━━━━━━━━━━━━━━━━ 0s 728us/step - loss: -345.3138
Epoch 10/10
347/347 ━━━━━━━━━━━━━━━━━━━━ 0s 676us/step - loss: -367.5252
347/347 ━━━━━━━━━━━━━━━━━━━━ 0s 822us/step
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[26]	validation's rmse: 11.3789
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[19]	validation's rmse: 3

...

337/337 ━━━━━━━━━━━━━━━━━━━━ 1s 657us/step - loss: -15.9609
Epoch 2/10
337/337 ━━━━━━━━━━━━━━━━━━━━ 0s 681us/step - loss: -88.6327
Epoch 3/10
337/337 ━━━━━━━━━━━━━━━━━━━━ 0s 737us/step - loss: -125.8951
Epoch 4/10
337/337 ━━━━━━━━━━━━━━━━━━━━ 0s 714us/step - loss: -193.8801
Epoch 5/10
337/337 ━━━━━━━━━━━━━━━━━━━━ 0s 683us/step - loss: -221.5029
Epoch 6/10
337/337 ━━━━━━━━━━━━━━━━━━━━ 0s 740us/step - loss: -247.4513
Epoch 7/10
337/337 ━━━━━━━━━━━━━━━━━━━━ 0s 815us/step - loss: -320.8503
Epoch 8/10
337/337 ━━━━━━━━━━━━━━━━━━━━ 0s 763us/step - loss: -369.2151
Epoch 9/10
337/337 ━━━━━━━━━━━━━━━━━━━━ 0s 740us/step - loss: -378.8726
Epoch 10/10
337/337 ━━━━━━━━━━━━━━━━━━━━ 0s 701us/step - loss: -462.2464
337/337 ━━━━━━━━━━━━━━━━━━━━ 0s 859us/step
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[20]	validation's rmse: 11.7067
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[15]	validation's rmse: 1

d:\Dropbox\Python\LaboIII\labo3-2024v\.venv\lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


320/320 ━━━━━━━━━━━━━━━━━━━━ 1s 679us/step - loss: -40.8308
Epoch 2/10
320/320 ━━━━━━━━━━━━━━━━━━━━ 0s 749us/step - loss: -177.1233
Epoch 3/10
320/320 ━━━━━━━━━━━━━━━━━━━━ 0s 700us/step - loss: -301.0370
Epoch 4/10
320/320 ━━━━━━━━━━━━━━━━━━━━ 0s 743us/step - loss: -396.8490
Epoch 5/10
320/320 ━━━━━━━━━━━━━━━━━━━━ 0s 757us/step - loss: -461.0225
Epoch 6/10
320/320 ━━━━━━━━━━━━━━━━━━━━ 0s 760us/step - loss: -545.6605
Epoch 7/10
320/320 ━━━━━━━━━━━━━━━━━━━━ 0s 741us/step - loss: -560.7589
Epoch 8/10
320/320 ━━━━━━━━━━━━━━━━━━━━ 0s 844us/step - loss: -655.9351
Epoch 9/10
320/320 ━━━━━━━━━━━━━━━━━━━━ 0s 727us/step - loss: -813.0811
Epoch 10/10
320/320 ━━━━━━━━━━━━━━━━━━━━ 0s 790us/step - loss: -871.2560
320/320 ━━━━━━━━━━━━━━━━━━━━ 0s 890us/step
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[64]	validation's rmse: 34.332
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[36]	validation's rmse: 3

d:\Dropbox\Python\LaboIII\labo3-2024v\.venv\lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


340/340 ━━━━━━━━━━━━━━━━━━━━ 1s 645us/step - loss: -8.5684
Epoch 2/10
340/340 ━━━━━━━━━━━━━━━━━━━━ 0s 710us/step - loss: -52.0508
Epoch 3/10
340/340 ━━━━━━━━━━━━━━━━━━━━ 0s 715us/step - loss: -86.0639
Epoch 4/10
340/340 ━━━━━━━━━━━━━━━━━━━━ 0s 720us/step - loss: -120.4294
Epoch 5/10
340/340 ━━━━━━━━━━━━━━━━━━━━ 0s 703us/step - loss: -156.0332
Epoch 6/10
340/340 ━━━━━━━━━━━━━━━━━━━━ 0s 738us/step - loss: -184.0946
Epoch 7/10
340/340 ━━━━━━━━━━━━━━━━━━━━ 0s 757us/step - loss: -214.3226
Epoch 8/10
340/340 ━━━━━━━━━━━━━━━━━━━━ 0s 730us/step - loss: -232.8817
Epoch 9/10
340/340 ━━━━━━━━━━━━━━━━━━━━ 0s 706us/step - loss: -254.6087
Epoch 10/10
340/340 ━━━━━━━━━━━━━━━━━━━━ 0s 806us/step - loss: -310.4522
340/340 ━━━━━━━━━━━━━━━━━━━━ 0s 840us/step
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[34]	validation's rmse: 9.11923
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[21]	validation's rmse: 6.2

In [7]:

predictions_all['tn']=predictions_all['tn'].astype('float32')
predictions_all.index.names = ['product_id']
predictions_all.to_csv(DATOS_DIR+'/pred/0025-prediccion-RSME_scaled_Robust-product_id_LSTM-weights2.csv', index=True,header=True)
print("Overall custom metric: ", average_metric)


Overall custom metric:  16.43546087366655
